<a href="https://colab.research.google.com/github/celiaferreira/Covid19_RX/blob/master/1_ImportarDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Importar dados de várias fontes
Uma aprendizagem eficiente permitirá distinguir os casos de **penumonia COVID-19** dos **restantes tipos de pneumonia** e das **situações normais**. 

Encontram-se na internet repositórios com as seguintes situações, que propomos distinguir no âmbito deste projeto:

* Covid-19
* Pneumonia viral
* Pneumonia bacteriana
* Situações sem patologia (normais)

Neste projeto foram recolhidos dados das seguintes fontes:

* https://www.kaggle.com/darshan1504/covid19-detection-xray-dataset
* https://www.kaggle.com/tawsifurrahman/covid19-radiography-database 
* https://github.com/ieee8023/covid-chestxray-dataset
* https://www.kaggle.com/theroyakash/covid19

### 1.1 Import packages

In [ ]:
from sklearn.utils.multiclass import unique_labels
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import expand_dims
import torch
import os
from tqdm import tqdm #barra de progreso`
import cv2
from PIL import Image
from glob import glob
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array
#para redimencionar
import sklearn
import skimage
from skimage.transform import resize
import random
from sklearn.metrics import confusion_matrix
import datetime
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from skimage.color import rgb2gray
from keras.utils import plot_model
from keras.models import Model,Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D, Flatten, Embedding, Dropout,BatchNormalization
from keras.layers import MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D,concatenate,Input
from keras.utils import to_categorical
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras import datasets, layers, models, optimizers
from keras.applications.imagenet_utils import preprocess_input
from imblearn.over_sampling import SMOTE
from collections import Counter
from keras.applications import VGG19, InceptionResNetV2, ResNet50

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/si

#### 1.2 Fontes online
* Kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "xxx" #incluir username json kaggle
os.environ['KAGGLE_KEY'] = "xxxxxxx" #incluir password json kaggle
!kaggle datasets download -d darshan1504/covid19-detection-xray-dataset
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database
!kaggle datasets download -d theroyakash/covid19

 94% 175M/186M [00:01<00:00, 116MB/s] 
100% 186M/186M [00:01<00:00, 116MB/s]
 99% 1.13G/1.15G [00:09<00:00, 123MB/s]
100% 1.15G/1.15G [00:09<00:00, 133MB/s]
 94% 121M/128M [00:01<00:00, 119MB/s]
100% 128M/128M [00:01<00:00, 124MB/s]


In [ ]:
from google.colab import files
import zipfile
with zipfile.ZipFile("/content/covid19-detection-xray-dataset.zip","r") as zip_ref:
    zip_ref.extractall("/content/covid19-detection-xray-dataset")
with zipfile.ZipFile("/content/covid19-radiography-database.zip","r") as zip_ref:
    zip_ref.extractall("/content/covid19-radiography-database")
with zipfile.ZipFile("/content/covid19.zip","r") as zip_ref:
    zip_ref.extractall("/content/covid19")

* Github

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 2467 (delta 79), reused 108 (delta 33), pack-reused 2283
Receiving objects: 100% (2467/2467), 511.75 MiB | 42.10 MiB/s, done.
Resolving deltas: 100% (1055/1055), done.
Checking out files: 100% (564/564), done.


#### 1.3 Definição de repositórios

In [ ]:
#Fonte 1: https://www.kaggle.com/darshan1504/covid19-detection-xray-dataset
PATH_train1 = '/content/covid19-detection-xray-dataset/NonAugmentedTrain/'
PATH_test1 ='/content/covid19-detection-xray-dataset/ValData/'
#/content/covid19-detection-xray-dataset/TrainData -> tem dados já aumentados, cópia de NonAugmentedTrain (não usado, porque faremos o nosso data augmentation)

#Fonte 2: https://www.kaggle.com/tawsifurrahman/covid19-radiography-database 
PATH2='/content/covid19-radiography-database/COVID-19 Radiography Database/'

#Fonte 3: https://github.com/ieee8023/covid-chestxray-dataset
PATH3='/content/covid-chestxray-dataset/'

#Fonte 4: https://www.kaggle.com/theroyakash/covid19 
PATH4= '/content/covid19/xrays/'


* Eliminar alguns ficheiros auxiliares que interferem com a importação

In [ ]:
os.remove("/content/covid19-radiography-database/COVID-19 Radiography Database/COVID-19.metadata.xlsx")
os.remove("/content/covid19-radiography-database/COVID-19 Radiography Database/NORMAL.metadata.xlsx")
os.remove("/content/covid19-radiography-database/COVID-19 Radiography Database/Viral Pneumonia.matadata.xlsx")
os.remove("/content/covid19-radiography-database/COVID-19 Radiography Database/README.md.txt")
os.remove("/content/covid-chestxray-dataset/SCHEMA.md")
os.remove("/content/covid-chestxray-dataset/README.md")
os.remove("/content/covid-chestxray-dataset/metadata.csv")
os.remove("/content/covid-chestxray-dataset/requirements.txt")

#### 1.4 Importar dados
A seguinte função auxilia na importação dos dados recolhidos dos vários repositórios, associando a respetiva label aos mesmos. Os dados são convertidos em arrays:

In [ ]:
def get_data(list):
    X = []
    y = []
    label=-1
    for folder in list:
        for folderName in os.listdir(folder):
            if not folderName.startswith('.'):
                if folderName in ['COVID-19','covid','images']:
                    label = 0
                elif folderName in ['ViralPneumonia','Viral Pneumonia','viral neumonia']:
                    label = 1
                elif folderName in ['BacterialPneumonia','Bacterial Pneumonia']:
                    label = 2               
                elif folderName in ['NORMAL','Normal','normal']:
                    label = 3
                for image_filename in tqdm(os.listdir(folder + folderName)):
                    img_file = cv2.imread(folder + folderName + '/' + image_filename)
                    if img_file is not None:
                        img_file = skimage.transform.resize(img_file, (200, 200, 3),mode='constant',
                                                            anti_aliasing=True)
                        img_file = rgb2gray(img_file)
                        img_arr = np.asarray(img_file)
                        X.append(img_arr)
                        y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)  
    return X,y

Na primeira invocação (LOAD_FROM_IMAGES = True) é feito o load das imagens para os arrays, que são guardados. Nas invocações seguintes (LOAD_FROM_IMAGES = False) é apenas feito o load dos arrays guardados.

In [ ]:
list=[PATH_train1,PATH_test1,PATH2,PATH3,PATH4]

LOAD_FROM_IMAGES = True
#LOAD_FROM_IMAGES = False
if LOAD_FROM_IMAGES:
    #converter imagens em arrays
    BD_X, BD_Y = get_data(list)

   #gravar os arrays em ficheiros binários numpy (.npy)
    np.save('x_BD.npy', BD_X)
    np.save('y_BD.npy',BD_Y)
    
else:
    #fazer o load dos arrays guardados
    X_all = np.load('x_BD.npy')
    y_all = np.load('y_BD.npy')  